In [7]:
using CSV,  DataFrames

periodo_anterior(x::Integer) =  x % 100 > 1  ?  x-1  : 12 + (div(x,100) -1) * 100

df = CSV.read("/home/gscm_5950/Downloads/competencia_03_crudo.csv", DataFrame)
sort!(df, [:numero_de_cliente, :foto_mes])

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad
,Int64,Int64,Int64,Int64,Int64,Int64
1,29183981,201901,1,0,1,48
2,29183981,201902,1,0,1,48
3,29183981,201903,1,0,1,48
4,29183981,201904,1,0,1,48
5,29183981,201905,1,0,1,48
6,29183981,201906,1,0,1,48
7,29183981,201907,1,0,1,48
8,29183981,201908,1,0,1,48
9,29183981,201909,1,0,1,48


In [8]:
global periodo_ultimo = maximum( df.foto_mes )
global periodo_anteultimo = periodo_anterior( periodo_ultimo)

# assign most common class values
df.clase_ternaria = @. ifelse( df.foto_mes < periodo_anteultimo, "CONTINUA", missing )

# pre compute sequential time
periodo = @. div(df.foto_mes,100)*12 + df.foto_mes%100

global last = nrow(df)

for i in 1:last
  if df.foto_mes[i] <= periodo_anteultimo &&  i < last && df.numero_de_cliente[i] != df.numero_de_cliente[i+1]
          df.clase_ternaria[i] = "BAJA+1"
  end

  if df.foto_mes[i] < periodo_anteultimo &&  i+1 < last && df.numero_de_cliente[i] == df.numero_de_cliente[i+1] &&  periodo[i+1] == periodo[i] +1 &&
        ( df.numero_de_cliente[i+1] != df.numero_de_cliente[i+2]  || df.numero_de_cliente[i+1] == df.numero_de_cliente[i+2] && periodo[i+2]  > periodo[i+1] +2)
          df.clase_ternaria[i] = "BAJA+2"
  end
end

CSV.write( "competencia_03_julia.csv", df )

"competencia_03_julia.csv"